# Import Library

In [1]:
import os
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import sklearn
from textblob import TextBlob
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from wordcloud import WordCloud
import time

# Import Dataset

> Variabel data 1 digunakan untuk skenario pelabelan 1
<br>
Variabel data 2 digunakan untuk skenario pelabelan 2

In [10]:
data1 = pd.read_csv('data bocor-new1.csv', sep=',', encoding='utf-8') 
data2 = pd.read_csv('data bocor-new1.csv', sep=',', encoding='utf-8') 

data1

,time,id,username,text
0,2021-11-21 23:56:14+00:00,1462570580775821313,BukanBuTejo,b'@txtdaritekno Gak usah sok keren data bocor ...
1,2021-11-21 16:35:53+00:00,1462459765494214661,Syan~GA ALBUM~CEK PINNED,b'@ameamakunai Bisa aja email kak ame bocor tr...
2,2021-11-21 14:52:08+00:00,1462433655008030720,Ivoox Indonesia,b'Data yang Bocor Berpotensi Disalahgunakan Pe...
3,2021-11-21 14:41:13+00:00,1462430907277529095,🦈 SUROBOYOFESS 🐊,b'Kok data lu bocor yaa?? Hmm isok ae lak mbuj...
4,2021-11-21 11:44:12+00:00,1462386359386222594,Aktual Official,b'CISSReC Sebut Data Pribadi Bocor Bisa Diguna...
...,...,...,...,...
204,2021-11-14 04:06:06+00:00,1459734360815386626,drama alter,"b'SUBES !! HALLO SUBES !! \n""Kebenaran akan da..."
205,2021-11-14 03:16:26+00:00,1459721858727612419,꧁jajansai~redi sealed memo20+POB꧂,b'@bonvoyajeon Tp kl aku kasih ripiw pasti aku...
206,2021-11-14 00:05:02+00:00,1459673694964379651,Jawa Pos,b'Teror Penagihan hingga Ancaman Data Pribadi ...
207,2021-11-13 17:09:33+00:00,1459569133096341506,CanduKenikmatan,"b'@rindubulanq @New_boyz_Sby tambahan tips, ka..."


In [11]:
# Hapus kolom yang tidak digunakan
# variabel data1 digunakan untuk skenario pelabelan 1
# variabel data2 digunakan untuk skenario pelabelan 2

data1 = data1.drop(data1.columns[[0,1,2]], axis=1)
data2 = data2.drop(data2.columns[[0,1,2]], axis=1)

# Skenario Pelabelan 1

In [14]:
# Pelabelan Dengan Textblob

def text_blop(txt):
    clean = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",txt).split())
    text = TextBlob(clean)
    
    try:
      text = text.translate(to="en")
      time.sleep(1)
    except Exception as e:
      print(e)

    if text.sentiment.polarity > 0.0:
      hasil = "Positif"
    
    elif text.sentiment.polarity == 0.0:
      hasil = "Netral"

    else:
      hasil = "Negatif"

    return hasil

data2['label'] = data2['text'].apply(lambda x: text_blop(x))
data2.head() 

Translation API returned the input string unchanged.
Translation API returned the input string unchanged.
Translation API returned the input string unchanged.
Translation API returned the input string unchanged.
Translation API returned the input string unchanged.
Translation API returned the input string unchanged.
Translation API returned the input string unchanged.


,text,label
0,b'@txtdaritekno Gak usah sok keren data bocor ...,Positif
1,b'@ameamakunai Bisa aja email kak ame bocor tr...,Positif
2,b'Data yang Bocor Berpotensi Disalahgunakan Pe...,Netral
3,b'Kok data lu bocor yaa?? Hmm isok ae lak mbuj...,Netral
4,b'CISSReC Sebut Data Pribadi Bocor Bisa Diguna...,Netral


## Preprocessing 

> all skenario preprocessing

In [24]:
# Removing Punctuationtrain

import string
import re
string.punctuation

def remove_punctuation(txt):
    '''a function for removing punctuation'''
    text = txt[1:]
    text = re.sub('@[^\s]+','',text)
    text = re.sub('http://\S+|https://\S+', '', text)
    text = re.sub("#\S+", " ", text)
    text = re.sub('\[.*?\]', '', str(text))
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('  ', '', text)
    return text

data1['clean'] = data1['text'].apply(lambda x: remove_punctuation(x))
data2['clean'] = data2['text'].apply(lambda x: remove_punctuation(x))

In [25]:
# Casefolding

def casefolding(txt):
    text = str(txt).lower()
    return text

data1['casefolding'] = data1['clean'].apply(lambda x: casefolding(x))
data2['casefolding'] = data2['clean'].apply(lambda x: casefolding(x))


In [26]:
#tokenizing
def tokenize(kalimat):
    text = word_tokenize(kalimat)
    return text

data1['tokenize'] = data1['casefolding'].apply(lambda x: tokenize(x))
data2['tokenize'] = data2['casefolding'].apply(lambda x: tokenize(x))

In [27]:
#filtering

def stopwords_removal(txt):
  filtering = stopwords.words('indonesian', 'english')
  x = []
  data = []
  def myFunc(x):
    if x in filtering:
      return False
    else:
      return True
  fit = filter(myFunc, txt)
  for x in fit:
    data.append(x)
  return data

data1['stopwords'] = data1['tokenize'].apply(lambda x: stopwords_removal(x))
data2['stopwords'] = data2['tokenize'].apply(lambda x: stopwords_removal(x))

In [ ]:
#stemming

from sklearn.pipeline import Pipeline
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(txt):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  do = []
  for w in txt:
    dt = stemmer.stem(w)
    do.append(dt)
  d_clean = []
  d_clean = " ".join(do)
  print(d_clean)
  return d_clean

data['stemming'] = data['stopwords'].apply(lambda x: stemming(x))

data.to_csv('data_clean.csv', index=False)

data_clean = pd.read_csv('data_clean.csv', encoding='latin1')
data_clean.head()